In [3]:
import tushare as ts
import datetime 
import tqdm
import time
pro = ts.pro_api("abc")
pro._DataApi__token = "c171s1b1d0ay2mr258"
pro._DataApi__http_url = "http://tushare.nlink.vip"

# from 20180101 to 20251210
date_str_list = [datetime.datetime.strftime(datetime.datetime(2018, 12, 14) + datetime.timedelta(days=i), "%Y%m%d") for i in range((datetime.datetime(2025, 12, 10) - datetime.datetime(2018, 12, 14)).days + 1)]

for date_str in tqdm.tqdm(date_str_list):
    for i in range(10000):
        try:
            df = pro.fut_daily(trade_date=date_str, exchange="CFFEX", fields="ts_code,trade_date,pre_close,pre_settle,open,high,low,close,settle,change1,change2,vol,amount,oi,oi_chg,delv_settle")
            break
        except Exception as e:
            #print(e)
            time.sleep(1)
    if len(df) > 1:
        df.to_csv(f"/root/sw1/tushare/daily_ohlcva/{date_str}.csv", index=False)


100%|██████████| 2554/2554 [12:01<00:00,  3.54it/s]  


In [4]:
import polars as pl
df = pl.scan_csv("./tushare/daily_ohlcva").collect()
df

ts_code,trade_date,pre_close,pre_settle,open,high,low,close,settle,change1,change2,vol,amount,oi,oi_chg,delv_settle
str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""IF.CFX""",20150105,3593.2,3579.8,3619.0,3768.8,3585.2,3671.4,3718.6,91.6,138.8,1.30381e6,1.4326e8,138633.0,null,null
"""IF1501.CFX""",20150105,3593.2,3579.8,3619.0,3768.8,3585.2,3671.4,3718.6,91.6,138.8,1.30381e6,1.4326e8,138633.0,9146.0,null
"""IF1502.CFX""",20150105,3635.2,3622.8,3645.0,3817.6,3636.0,3726.8,3766.0,104.0,143.2,31115.0,3.4637e6,10631.0,2486.0,null
"""IF1503.CFX""",20150105,3672.4,3656.2,3681.0,3860.2,3677.4,3766.0,3807.8,109.8,151.6,117300.0,1.3202e7,60335.0,8590.0,null
"""IF1506.CFX""",20150105,3695.2,3685.2,3710.0,3902.0,3708.0,3801.8,3845.6,116.6,160.4,48418.0,5.5061e6,27622.0,1562.0,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""TS2603.CFX""",20251210,102.43,102.42,102.424,102.468,102.416,102.456,102.46,0.036,0.04,36879.0,7.5563e6,70758.0,5764.0,null
"""TS2606.CFX""",20251210,102.444,102.438,102.442,102.474,102.434,102.466,102.468,0.028,0.03,310.0,63520.228,2550.0,-6.0,null
"""TSL.CFX""",20251210,102.462,102.462,102.45,102.49,102.422,102.476,102.476,0.014,0.014,704.0,144266.316,915.0,null,null


In [5]:
IC = df.filter((pl.col("ts_code").str.starts_with("IC")) & (~pl.col("ts_code").is_in(["IC.CFX", "ICL.CFX", "ICL1.CFX", "ICL2.CFX", "ICL3.CFX"])))
IC

ts_code,trade_date,pre_close,pre_settle,open,high,low,close,settle,change1,change2,vol,amount,oi,oi_chg,delv_settle
str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""IC1505.CFX""",20150416,null,7818.6,7681.0,7790.0,7480.2,7707.8,7642.8,-110.8,-175.8,116888.0,1.7929e7,6643.0,null,null
"""IC1506.CFX""",20150416,null,7830.6,7520.0,7757.0,7464.4,7652.4,7597.8,-178.2,-232.8,5285.0,805699.368,856.0,null,null
"""IC1509.CFX""",20150416,null,7866.8,7752.8,7762.8,7516.2,7643.2,7604.0,-223.6,-262.8,743.0,113438.296,215.0,null,null
"""IC1512.CFX""",20150416,null,7928.8,7799.6,7799.6,7525.0,7625.2,7571.0,-303.6,-357.8,484.0,73962.88,226.0,null,null
"""IC1505.CFX""",20150417,7707.8,7642.8,7722.0,7918.8,7631.2,7758.2,7803.4,115.4,160.6,114214.0,1.7814e7,9144.0,2501.0,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""IC2606.CFX""",20251209,6758.4,6766.4,6747.8,6776.0,6702.0,6711.6,6712.0,-54.8,-54.4,6800.0,916346.476,34127.0,328.0,null
"""IC2512.CFX""",20251210,7094.4,7094.2,7083.0,7143.6,7040.0,7122.2,7128.2,28.0,34.0,69706.0,9.8795e6,109494.0,-4521.0,null
"""IC2601.CFX""",20251210,7038.8,7037.6,7024.0,7083.2,6985.6,7065.8,7066.8,28.2,29.2,7949.0,1.1177e6,19026.0,2737.0,null


In [6]:
IM = df.filter((pl.col("ts_code").str.starts_with("IM")) & (~pl.col("ts_code").is_in(["IM.CFX", "IML.CFX", "IML1.CFX", "IML2.CFX", "IML3.CFX"])))
IM

ts_code,trade_date,pre_close,pre_settle,open,high,low,close,settle,change1,change2,vol,amount,oi,oi_chg,delv_settle
str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""IM2208.CFX""",20220722,null,7018.8,7050.0,7085.2,6857.0,6956.6,6934.2,-62.2,-84.6,22153.0,3.0919e6,17517.0,null,null
"""IM2209.CFX""",20220722,null,6988.8,6987.0,7043.2,6807.0,6901.2,6877.4,-87.6,-111.4,6710.0,930226.732,5858.0,null,null
"""IM2212.CFX""",20220722,null,6898.8,6895.8,6946.8,6667.6,6737.0,6719.4,-161.8,-179.4,3921.0,533484.84,3613.0,null,null
"""IM2303.CFX""",20220722,null,6798.8,6800.0,6838.6,6530.0,6588.0,6572.4,-210.8,-226.4,3703.0,493444.812,3338.0,null,null
"""IM2208.CFX""",20220725,6956.6,6934.2,6974.2,7015.8,6850.0,6878.6,6885.8,-55.6,-48.4,22883.0,3.1700e6,18747.0,1230.0,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""IM2606.CFX""",20251209,6900.2,6904.6,6890.0,6921.4,6861.4,6869.4,6870.6,-35.2,-34.0,11357.0,1.5646e6,59887.0,981.0,null
"""IM2512.CFX""",20251210,7347.8,7352.0,7340.8,7390.2,7295.0,7371.4,7374.6,19.4,22.6,108100.0,1.5863e7,163906.0,-1039.0,null
"""IM2601.CFX""",20251210,7270.6,7274.2,7255.0,7308.8,7220.6,7290.0,7293.8,15.8,19.6,10859.0,1.5766e6,27271.0,3862.0,null


In [7]:
IC.write_csv("raw_data/IC.csv")
IM.write_csv("raw_data/IM.csv")